In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import calendar
from time import sleep
from datetime import datetime
from zoneinfo import ZoneInfo
import json

from helpers.utils import normalize_name

In [53]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'
response = requests.get(url)
html = response.text.replace('<!--', '').replace('-->', '')
soup = BeautifulSoup(html, 'lxml')

In [54]:
with open('utils/rotowire_tm_map.json', 'r') as f:
    tm_map = json.load(f)

In [ ]:
tm = 'MIL'

home_tags = soup.find_all('a', class_ = 'lineup__team is-home')
away_tags = soup.find_all('a', class_ = 'lineup__team is-visit')

home_lineup_tags = soup.find_all('ul', class_ = 'lineup__list is-home')
away_lineup_tags = soup.find_all('ul', class_ = 'lineup__list is-visit')

for home_tag, away_tag, home_lineup_tag, away_lineup_tag in zip(
    home_tags, away_tags, home_lineup_tags, away_lineup_tags
) :
    
    rw_home = home_tag.find(class_ = 'lineup__abbr').text
    rw_away = away_tag.find(class_ = 'lineup__abbr').text

    home, away = tm_map[rw_home], tm_map[rw_away]

    if home == tm or away == tm :

        starting_lineup_tags = home_lineup_tag.find_all('li')[1:6]
        players_home = [' '.join(x.find('a', href = True)['href'].split('/')[-1].split('-')[:-1]) for x in starting_lineup_tags]
        df_home = pd.DataFrame(players_home, columns = ['name'])
        df_home['name_norm'] = df_home['name'].apply(lambda x: normalize_name(x, True))
        df_home['Team'] = home

        starting_lineup_tags = away_lineup_tag.find_all('li')[1:6]
        players_away = [' '.join(x.find('a', href = True)['href'].split('/')[-1].split('-')[:-1]) for x in starting_lineup_tags]
        df_away = pd.DataFrame(players_away, columns = ['name'])
        df_away['name_norm'] = df_away['name'].apply(lambda x: normalize_name(x, True))
        df_away['Team'] = away
        break

    else :
        pass

In [1]:
from helpers.scrape import get_rotowire_lineups, get_lineups

In [4]:
soup = get_rotowire_lineups()
get_lineups(soup, 'ORL')

,name,name_norm,Team
0,jalen suggs,jalensuggs,ORL
1,kentavious caldwell pope,kentaviouscaldwellpope,ORL
2,tristan da silva,tristandasilva,ORL
3,wendell carter,wendellcarter,ORL
4,goga bitadze,gogabitadze,ORL
5,jalen brunson,jalenbrunson,NYK
6,mikal bridges,mikalbridges,NYK
7,josh hart,joshhart,NYK
8,og anunoby,oganunoby,NYK
9,karl anthony towns,karlanthonytowns,NYK


In [22]:
metadata = pd.read_csv('data/player_metadata.csv')[['name_norm', 'player_id']]
metadata['name_norm'] = metadata['name_norm'].str.replace(' ', '')

In [28]:
lineups = lineups.merge(metadata,
              on = 'name_norm',
              how = 'left')

In [6]:
import pandas as pd

In [16]:
odds = pd.read_csv('data/odds_first_basket.csv')[['name']].drop_duplicates()
odds['name_norm'] = odds['name'].apply(normalize_name)
metadata = pd.read_csv('data/player_metadata.csv')[['name_norm', 'player_id']]

In [18]:
merged = odds.merge(metadata, on = 'name_norm', how = 'left')

In [19]:
merged[merged['player_id'].isna()]

,name,name_norm,player_id
83,Nicolas Claxton,nicolas claxton,NaN
121,Herb Jones,herb jones,NaN


In [27]:
roster_cols = ['MP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GmSc','+/-','TS%','eFG%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg','BPM','VORP']

In [34]:
lineups[roster_cols] = np.ones((lineups.shape[0], len(roster_cols))) * np.nan
lineups['starter'] = True

In [35]:
lineups.insert(0, 'game_id', '202412140MIL')

In [37]:
from helpers.preprocess import feature_engineering

In [44]:
data, features = feature_engineering(lineups.drop(columns = 'name_norm').rename(columns = {'name': 'Player'}))

In [46]:
data_game = data.copy()[data['game_id'] == '202412140MIL']

In [47]:
data_game

,game_id,Player,player_id,first_basket_scorer,PTS_avg,PTS_5,PTS_25,PTS_50,USG%_avg,USG%_5,...,VORP_50,FGA_avg,FGA_5,FGA_25,FGA_50,first_basket_avg,first_basket_5,first_basket_25,first_basket_50,rating
123531,202412140MIL,shai gilgeous alexander,gilgesh01,NaN,1.707934,2.106276,2.047717,1.908537,1.484137,1.535314,...,1.765375,1.289645,1.630126,1.643707,1.580751,0.667548,0.707102,0.138706,-0.177875,NaN
123532,202412140MIL,cason wallace,wallaca01,NaN,-1.427768,-0.820627,-1.299398,-1.334137,-1.622787,-0.825841,...,-0.646620,-1.443730,-1.025697,-1.260003,-1.276426,-1.737972,-0.707102,-1.248357,-1.363712,NaN
123533,202412140MIL,luguentz dort,dortlu01,NaN,-0.345258,-1.066815,-0.928824,-0.800015,-0.420689,-1.155501,...,-0.887231,-0.221981,-0.979907,-0.966175,-0.854672,-0.099441,-0.707102,-0.323648,-0.177875,NaN
123534,202412140MIL,jalen williams,willija06,NaN,0.559758,0.930044,0.804503,0.676846,0.262599,1.132396,...,0.693564,0.396581,1.218015,0.814076,0.654546,-0.004110,0.707102,-0.323648,-0.474335,NaN
123535,202412140MIL,isaiah hartenstein,harteis01,NaN,-1.558142,-0.930044,-0.677791,-0.951787,-1.259332,-0.699048,...,0.430239,-1.809919,-1.117277,-1.121731,-1.313640,-1.591275,0.707102,-0.323648,-0.177875,NaN
123536,202412140MIL,fred vanvleet,vanvlfr01,NaN,0.136264,0.109417,0.242666,0.192342,0.072767,0.137781,...,0.295631,0.318711,0.622745,0.572100,0.418860,-0.361400,-0.707102,-0.786003,-0.770794,NaN
123537,202412140MIL,jalen green,greenja05,NaN,1.031715,0.355605,0.499676,0.831537,1.208954,0.690032,...,-0.411898,1.247484,0.256424,0.779508,1.068030,0.707921,-0.707102,0.138706,1.600879,NaN
123538,202412140MIL,dillon brooks,brookdi01,NaN,-0.045083,-0.382959,-0.492504,-0.636568,0.249017,-0.614520,...,-1.588030,0.313788,-0.567797,-0.343951,-0.420514,0.891846,-0.707102,-0.323648,-0.770794,NaN
123539,202412140MIL,jabari smith,smithja05,NaN,-0.250273,-0.738564,-0.635952,-0.408910,-0.620913,-1.090696,...,-0.591094,-0.115072,-0.659377,-0.689631,-0.420514,0.317082,-0.707102,0.601061,0.711502,NaN
123540,202412140MIL,alperen sengun,sengual01,NaN,0.190853,0.437668,0.439907,0.522155,0.646247,0.890082,...,0.940064,0.024493,0.622745,0.572100,0.563579,1.209800,2.121305,2.450479,1.600879,NaN


In [48]:
import pickle
X = data_game[features].to_numpy()

# Load the model from the .pkl file
with open('models/model_rf.pkl', "rb") as f:
    model = pickle.load(f)

In [49]:
y_pred = model.predict_proba(X)[:, -1]

In [51]:
data_game = data_game[['game_id', 'player_id', 'Player']]
data_game['Pred. prob (%)'] = y_pred
data_game['Pred. prob (%)'] = data_game['Pred. prob (%)'] / data_game.groupby('game_id')['Pred. prob (%)'].transform('sum')
data_game['Pred. odds'] = data_game['Pred. prob (%)'].apply(lambda x: round(1/x, 1))
data_game['Pred. prob (%)'] = data_game['Pred. prob (%)'].apply(lambda x: round(x * 100, 1))
data_game = data_game.sort_values(['game_id', 'Pred. odds']).reset_index(drop = True)

/var/folders/yb/vy0d2g3s36j5mzjxqc1kwwpw0000gr/T/ipykernel_94047/771665105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_game['Pred. prob (%)'] = y_pred
/var/folders/yb/vy0d2g3s36j5mzjxqc1kwwpw0000gr/T/ipykernel_94047/771665105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_game['Pred. prob (%)'] = data_game['Pred. prob (%)'] / data_game.groupby('game_id')['Pred. prob (%)'].transform('sum')
/var/folders/yb/vy0d2g3s36j5mzjxqc1kwwpw0000gr/T/ipykernel_94047/771665105.py:4: SettingWithCopyW

In [52]:
data_game

,game_id,player_id,Player,Pred. prob (%),Pred. odds
0,202412140MIL,sengual01,alperen sengun,13.6,7.3
1,202412140MIL,gilgesh01,shai gilgeous alexander,12.4,8.1
2,202412140MIL,greenja05,jalen green,11.9,8.4
3,202412140MIL,willija06,jalen williams,10.7,9.4
4,202412140MIL,vanvlfr01,fred vanvleet,9.7,10.4
5,202412140MIL,brookdi01,dillon brooks,9.5,10.5
6,202412140MIL,smithja05,jabari smith,9.2,10.9
7,202412140MIL,harteis01,isaiah hartenstein,8.0,12.4
8,202412140MIL,dortlu01,luguentz dort,7.9,12.7
9,202412140MIL,wallaca01,cason wallace,7.2,13.9


In [24]:
home, away = tm_map[rw_home], tm_map[rw_away]

In [46]:
lineup_away = soup.find('ul', class_ = 'lineup__list is-visit')
starting_lineup_tags = lineup_away.find_all('li')[1:6]
players_away = [' '.join(x.find('a', href = True)['href'].split('/')[-1].split('-')[:-1]) for x in starting_lineup_tags]
df_away = pd.DataFrame(players_away, columns = ['name'])
df_away['name_norm'] = df_away['name'].apply(normalize_name)
df_away['team'] = away
df_away

,name,name_norm,team
0,trae young,trae young,ATL
1,dyson daniels,dyson daniels,ATL
2,zaccharie risacher,zaccharie risacher,ATL
3,jalen johnson,jalen johnson,ATL
4,clint capela,clint capela,ATL


In [47]:
lineup_home = soup.find('ul', class_ = 'lineup__list is-home')
starting_lineup_tags = lineup_home.find_all('li')[1:6]
players_home = [' '.join(x.find('a', href = True)['href'].split('/')[-1].split('-')[:-1]) for x in starting_lineup_tags]
df_home = pd.DataFrame(players_home, columns = ['name'])
df_home['name_norm'] = df_home['name'].apply(normalize_name)
df_home['team'] = home
df_home

,name,name_norm,team
0,damian lillard,damian lillard,MIL
1,andre jackson,andre jackson,MIL
2,taurean prince,taurean prince,MIL
3,giannis antetokounmpo,giannis antetokounmpo,MIL
4,brook lopez,brook lopez,MIL


In [7]:
lineup_home = soup.find('ul', class_ = 'lineup__list is-home')
starting_lineup_tags = lineup_home.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['D. Lillard', 'Andre Jackson', 'T. Prince', 'G. Antetokounmpo', 'Brook Lopez']

In [48]:
[x for x in starting_lineup_tags.find_all('a', href = True)]

AttributeError: 'list' object has no attribute 'find_all'

In [50]:
[x['href'] for x in lineup_home.find_all('a', href = True)]

['/basketball/player/cade-cunningham-5336',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/tobias-harris-3211',
 '/basketball/player/jalen-duren-5703',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/bobi-klintman-5976',
 '/basketball/player/ausar-thompson-5932']

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.array([[0,1], [2, 3]]), columns = ['col1', 'col2'])

In [3]:
# Specify the email contents
mail = MIMEMultipart()
html = """\
<html><head></head><body>{0}</body></html>
""".format(build_table(df, 'grey_light', text_align = 'right', font_family = 'arial', width_dict = ['100','200','200','100','100','100','100'], font_size = 10))
mail.attach(MIMEText(html, 'html'))


In [4]:

# Set my email address and the password key
my_mail  = 'martinbog19@gmail.com'
# with open('gmail_key.txt') as f:
#     password = f.read()

# Set the subject of the email
mail['Subject'] = 'test_df'


In [ ]:

# Send the email (to myself)
server.sendmail(my_mail, my_mail, mail.as_string())
q = server.quit()

Email sent successfully!


In [9]:
import os

def is_running_in_github_actions():
    return os.getenv("GITHUB_ACTIONS") == "true"

In [12]:
os.getenv("k")

In [13]:
import pandas as pd
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import requests
import os
import json



if os.getenv("GITHUB_ACTIONS") == "true" :
  api_key = os.getenv('ODDS_API_KEY')
else :
  with open('secrets/odds_api_key.txt') as f:
    api_key = f.read()

with open('utils/odds_tm_map.json', 'r') as f :
  odds_tm_map = json.load(f)

games = pd.read_csv('data/games.csv')

# Store subset of games in the next 30 minutes
now = datetime.now(ZoneInfo('America/New_York'))
# games_now = games[(games['Time'] > now.strftime('%Y-%m-%d %H:%M:%S')) & (games['Time'] <= (now + timedelta(minutes = 30)).strftime('%Y-%m-%d %H:%M:%S'))]


games_now = games.tail(1)

# Iterate through games
for _, game in games_now.iterrows() :

    eventId = game['event_id']

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events/{eventId}/odds',
                             params = {'apiKey': api_key,
                                       'regions': 'us',
                                       'markets': 'player_first_basket',
                                       'oddsFormat': 'decimal'})
    

    bm_dfs = [pd.DataFrame(columns = ['name', 'price', 'bookmaker', 'update_time'])]
    for bookmaker in odds_response.json()['bookmakers'] :
        
        bm_df = pd.DataFrame(bookmaker['markets'][0]['outcomes'])
        bm_df['bookmaker'] = bookmaker['key']
        bm_df['update_time'] = bookmaker['markets'][0]['last_update']

        bm_dfs.append(
            bm_df
            .drop(columns = 'name')
            .rename(columns = {'description': 'name'})
        )

    game_df = pd.concat(bm_dfs).reset_index(drop = True)
    game_df['game_id'] = game['game_id']
    game_df['event_id'] = game['event_id']

    game_df['insert_timestamp_utc'] = datetime.now(timezone.utc)
    game_df.to_csv('data/odds_first_basket.csv', index = None, header = None, mode = 'a')

In [15]:
odds = pd.read_csv('data/odds_first_basket.csv')

In [21]:
odds[odds['game_id'] == '202412050GSW'].groupby('name').agg({'price': ['mean', 'min', 'max']})

price            
                         mean   min   max
name                                     
Alperen Sengun       6.375000   6.0   6.5
Andrew Wiggins       6.625000   6.5   7.0
Brandin Podziemski  10.875000   9.5  12.0
Buddy Hield          8.000000   8.0   8.0
Dillon Brooks        9.375000   9.0  10.5
Fred VanVleet        9.875000   9.5  10.0
Jabari Smith Jr     10.000000   9.5  11.0
Jalen Green          7.625000   7.0   8.0
Jonathan Kuminga     7.500000   6.5   8.5
Kevon Looney         8.625000   8.0  10.5
Moses Moody         12.333333  12.0  13.0

In [47]:
first_basket = pd.read_csv('data/first_basket_2025.csv')
games = pd.read_csv('data/games.csv')
rosters = pd.read_csv('data/rosters.nosync/rosters_2025.csv')
odds = pd.read_csv('data/odds_first_basket.csv')

In [48]:
game = rosters[(rosters['game_id'] == '202412140OKC') & rosters['starter']]
odds = odds[(odds['bookmaker'] == 'fanduel') & (odds['game_id'] == '202412140OKC')]

In [54]:
game = (
    game[['game_id', 'Player', 'player_id', 'Team']]
    .merge(first_basket[['game_id', 'first_basket']],
           on = 'game_id',
           how = 'left')
    .merge(odds[['price', 'player_id']],
           on = 'player_id',
           how = 'left')
)

game['first_basket_scorer'] = (game['player_id'] == game['first_basket']).astype(int)

In [60]:
game['stake'] = np.random.uniform(0, 10, (10,))
game['return'] = game['stake'] * (game['price'] * game['first_basket_scorer'] - 1)
game

,game_id,Player,player_id,Team,first_basket,price,first_basket_scorer,stake,return
0,202412140OKC,Jalen Williams,willija06,OKC,sengual01,8.0,0,3.059645,-3.059645
1,202412140OKC,Shai Gilgeous-Alexander,gilgesh01,OKC,sengual01,5.2,0,7.784887,-7.784887
2,202412140OKC,Isaiah Hartenstein,harteis01,OKC,sengual01,9.5,0,1.148188,-1.148188
3,202412140OKC,Cason Wallace,wallaca01,OKC,sengual01,11.0,0,6.010185,-6.010185
4,202412140OKC,Luguentz Dort,dortlu01,OKC,sengual01,13.0,0,2.718626,-2.718626
5,202412140OKC,Fred VanVleet,vanvlfr01,HOU,sengual01,10.0,0,0.128467,-0.128467
6,202412140OKC,Jabari Smith Jr.,smithja05,HOU,sengual01,11.0,0,0.768070,-0.768070
7,202412140OKC,Jalen Green,greenja05,HOU,sengual01,7.5,0,7.809358,-7.809358
8,202412140OKC,Dillon Brooks,brookdi01,HOU,sengual01,13.0,0,1.628389,-1.628389
9,202412140OKC,Alperen Sengun,sengual01,HOU,sengual01,6.0,1,5.985938,29.929689


In [59]:
import numpy as np

In [2]:
from datetime import datetime
import pandas as pd

In [3]:
from helpers.scrape import get_rotowire_lineups, get_lineups

In [4]:
rotowire_soup = get_rotowire_lineups()
rw_t = datetime.now()

In [5]:
game_id = '202412160CLE'

lineups = get_lineups(rotowire_soup, game_id[-3:])

In [7]:
lineups['game_id'] = game_id

In [9]:
lineups['metadata_insert_timestamp'] = rw_t

In [12]:
lineups.head(0).to_csv('data/rotowire_lineups.csv', index = None)

In [1]:
import pandas as pd

In [2]:
from helpers.scrape import get_players

In [5]:
data = get_players(2025)

In [14]:
data[
    (data.groupby('player_id')['player_id'].transform('count') == 1) # Only one team
    | data['Team'].str.match(r'^\d+TM$')                             # Total row
]

,Rk,name,Age,Team,Pos,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,name_norm,player_id
0,1,Giannis Antetokounmpo,30,MIL,PF,26,26,35.1,12.7,21.0,...,11.7,6.0,0.8,1.5,3.3,2.5,32.4,NaN,giannis antetokounmpo,antetgi01
1,2,Shai Gilgeous-Alexander,26,OKC,PG,33,33,34.4,11.0,20.8,...,5.5,6.1,2.0,1.1,2.6,2.1,31.2,NaN,shai gilgeousalexander,gilgesh01
2,3,Nikola Jokić,29,DEN,C,29,29,36.9,11.7,20.8,...,13.0,9.7,1.7,0.6,3.3,2.0,30.7,NaN,nikola jokic,jokicni01
3,4,LaMelo Ball,23,CHO,PG,22,22,33.7,10.5,24.6,...,5.3,7.4,1.3,0.2,4.1,3.7,30.1,NaN,lamelo ball,ballla01
4,5,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,...,8.8,5.6,0.6,0.8,2.2,2.6,29.0,NaN,paolo banchero,banchpa01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,508,Mason Jones,26,SAC,SG,3,0,4.3,0.0,0.7,...,1.3,2.3,0.0,0.0,0.0,0.3,0.0,NaN,mason jones,jonesma05
522,509,Mac McClung,26,ORL,SG,1,0,5.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,mac mcclung,mccluma01
523,510,Justin Minaya,25,POR,SF,1,0,6.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,justin minaya,minayju01
524,511,Riley Minix,24,SAS,SF,1,0,7.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,riley minix,minixri01


In [8]:
data.groupby('player_id')['player_id'].transform('count')

0      1
1      1
2      1
3      1
4      1
      ..
521    1
522    1
523    1
524    1
525    1
Name: player_id, Length: 526, dtype: int64

In [11]:
data['Team'].str.match(r'^\d+TM$').head(100).tail(20)

80    False
81    False
82    False
83     True
84    False
85    False
86    False
87    False
88    False
89    False
90    False
91    False
92    False
93    False
94    False
95    False
96    False
97    False
98    False
99    False
Name: Team, dtype: bool

In [40]:
import pandas as pd
import json
from helpers.utils import normalize_name

In [75]:
with open('utils/name_map_odds.json', 'r') as f:
    name_map = json.load(f)

In [85]:
odds = pd.read_csv('data/odds_first_basket.csv')[['name']].drop_duplicates().sort_values('name').reset_index(drop = True)
metadata = pd.read_csv('data/player_metadata.csv')
odds['name_norm'] = odds['name'].apply(normalize_name)
assert odds['name_norm'].value_counts().max() == 1
odds['player_id'] = odds['name'].map(name_map)
assert odds['player_id'].value_counts().max() == 1
# Subset of players needing to be mapped
odds_ = odds.copy()[odds['player_id'].isna()]

In [91]:
merged = pd.merge(
    odds_[['name', 'name_norm']],
    metadata[['name_norm', 'player_id']],
    on = 'name_norm',
    how = 'left'
    )

merged_check = merged.copy()[merged['player_id'].notna()]

namde_map_add = dict(zip(merged_check['name'], merged_check['player_id']))

In [93]:
with open('utils/name_map_odds.json', 'w') as f:
    json.dump(name_map | namde_map_add, f, indent = 4)

In [ ]:
 "Zach LaVine": "lavinza01",

"Herb Jones": "joneshe01",
    "Nicolas Claxton": "claxtni01"

In [94]:
merged_need = merged.copy()[merged['player_id'].isna()]

In [98]:
players_need_id = merged_need['name'].to_list()

In [113]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
# from pretty_html_table import build_table
import os

# Specify the email contents
player_list = '\n\t'.join(players_need_id)
indent = '\n\n\t'

mail = MIMEText(
f"""
{len(players_need_id)} players could not be matched to a Basketball Reference ID:
{indent}{player_list}
"""
)

# mail.attach('hi')

# Set my email address and the password key
my_mail = 'martinbog19@gmail.com'
if os.getenv("GITHUB_ACTIONS") == "true" :
    app_password = os.getenv('GMAIL_APP_KEY')
else :
    with open('secrets/gmail_app_key.txt') as f:
        app_password = f.read()
# Set the subject of the email
mail['Subject'] = '[LeFirstBasket] Manual name-ID matching required (odds)'

# Send the email using Gmail's SMTP server
with smtplib.SMTP("smtp.gmail.com", 587) as server:
    server.starttls()  # Upgrade connection to secure
    server.login(my_mail, app_password)  # Login with app password
    server.sendmail(my_mail, ['martinbog19@gmail.com'], mail.as_string())
print("Email sent successfully!")

Email sent successfully!
